In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
import json
import numpy as np
import pandas as pd
import keras
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [ ]:
x_train = np.load('/content/drive/My Drive/data.npy')
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
y_train = np.load('/content/drive/My Drive/lable.npy')
y_train = keras.utils.to_categorical(y_train, num_classes = 2)

In [ ]:
print(x_train.shape)
print(y_train.shape)

(305, 224, 224, 3)
(305, 2)


In [ ]:
x_train = x_train.astype('float32')
x_train /= 255

In [ ]:
densenet = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
model = Sequential()
model.add(densenet)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

29089792/29084464 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2050      
Total params: 7,039,554
Trainable params: 6,955,906
Non-trainable params: 83,648
_________________________________________________________________
None


In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=100,
    validation_split=0.1
)


Epoch 1/100
5/5 [==============================] - 149s 30s/step - loss: 0.9197 - accuracy: 0.5693 - val_loss: 1.7574 - val_accuracy: 0.3548
Epoch 2/100
5/5 [==============================] - 154s 31s/step - loss: 0.2331 - accuracy: 0.9051 - val_loss: 0.4093 - val_accuracy: 0.8710
Epoch 3/100
5/5 [==============================] - 151s 30s/step - loss: 0.3431 - accuracy: 0.8686 - val_loss: 0.7225 - val_accuracy: 0.9355
Epoch 4/100
5/5 [==============================] - 153s 31s/step - loss: 0.1917 - accuracy: 0.9234 - val_loss: 0.5898 - val_accuracy: 0.9355
Epoch 5/100
5/5 [==============================] - 152s 30s/step - loss: 0.1283 - accuracy: 0.9416 - val_loss: 0.3967 - val_accuracy: 0.9355
Epoch 6/100
5/5 [==============================] - 152s 30s/step - loss: 0.0648 - accuracy: 0.9708 - val_loss: 0.3615 - val_accuracy: 0.9355
Epoch 7/100
5/5 [==============================] - 149s 30s/step - loss: 0.0953 - accuracy: 0.9635 - val_loss: 1.3857 - val_accuracy: 0.1613
Epoch 8/100
5

In [9]:
model.save_weights("densenet_weights.h5")

In [10]:
!ls

densenet_weights.h5  drive  sample_data


In [11]:
from google.colab import files
files.download('densenet_weights.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
x_test = np.load('/content/drive/My Drive/test_images.npy')
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)
y_test = np.load('/content/drive/My Drive/test_labels.npy')
x_test = x_test.astype('float32')
x_test /= 255
x_test.shape

(98, 224, 224, 3)

In [13]:
result = model.predict_classes(x_test)
print(result)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
